In [1]:
# try to load a model

from transformers import AutoModelForCausalLM, AutoTokenizer
import yaml
import torch
# from coconut.coconut import Coconut
from coconut.utils import Config, set_seed, ProgressCallbackNoPrint, rm_old_prog_cb


## Load

In [7]:
from coconut.coconut import (
    CoconutConfig,
    CoconutQwen2ForCausalLM,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
f = "../outputs/gsm-qwen_20250201-122443/checkpoint_3"
fc = f + '/config.yaml'
with open(fc) as f2:
    config_dict = yaml.safe_load(f2)
configs = Config(config_dict)

model = CoconutQwen2ForCausalLM.from_pretrained(f, device_map=device)
tokenizer = AutoTokenizer.from_pretrained(f)
# model.load_state_dict(safe_open(f1, 'pt'))

## Eval

In [8]:
from coconut.dataset import (
    CoconutCollator,
    get_cot_latent_dataset,
    get_dataset,
    get_question_only_latent_dataset,
)
from coconut.eval import evaluate

bot_id = tokenizer.convert_tokens_to_ids("<|start-latent|>")
eot_id = tokenizer.convert_tokens_to_ids("<|end-latent|>")
scheduled_stage = 3
latent_id = tokenizer.convert_tokens_to_ids("<|latent|>")

max_size = 1024
base_dataset_valid = get_dataset(
    '../' + configs.val_path, tokenizer, max_size=max_size//30+3, drop_unused=False
)

dataset_gen_val = get_question_only_latent_dataset(
    scheduled_stage,
    base_dataset_valid,
    configs,
    bot_id,
    latent_id,
    eot_id,
    no_bot_eot=False,
    # drop_unused=False,
)
collator = CoconutCollator(tokenizer, latent_id=latent_id, label_pad_token_id=-100)
valid_gen_dataloader = torch.utils.data.DataLoader(
    dataset_gen_val,
    num_workers=1,
    pin_memory=True,
    batch_size=1,
    collate_fn=collator,
)

# run eval
max_new_tokens = 64
device = "cuda"
dtype = torch.bfloat16
phase = -1

r = evaluate(valid_gen_dataloader, model, tokenizer, base_dataset_valid, max_new_tokens=max_new_tokens, name=f"eval_{phase}", dtype=dtype, device=device)

../data/gsm_valid.json (num_proc=32):   0%|          | 0/37 [00:00<?, ? examples/s]

q_latent_3 (num_proc=32):   0%|          | 0/37 [00:00<?, ? examples/s]

Test Accuracy eval_-1:   0%|          | 0/37 [00:00<?, ?it/s]

2025-02-01 14:21:41.550 | INFO     | coconut.eval:evaluate:24 - Starting evaluation eval_-1
2025-02-01 14:21:43.164 | INFO     | coconut.eval:evaluate:67 - Q #0: Answer = '300' ideal_CoT = '<<4-2=2>>
	<<2/.5=4>>
	<<12/4=3>>
	<<100*3=300>>,'.
Question: `John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year?`.
Extracted llm Output: `1600` (=? 300) ❌.
Full llm output: `John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year?
	<|start-latent|><|latent|><|latent|><|latent|><|latent|><|latent|><|latent|><|end-latent|><<16*100=1600>>
	### 1600`. 

2025-02-01 14:21:43.402 | INFO     | coconut.eval:evaluate:67 - Q #1: Answer = '10' ideal_CoT = '<<1.5*2=3>>
	<<3+2.5=5.5>>
	<<1.5+3+5.5=10>>,'.
Question: `Hannah has three dogs. Th